In [235]:
df1

,person_income,person_home_ownership,loan_amnt,loan_int_rate,loan_percent_income,credit_score,loan_status,previous_loan_defaults_on_file_Yes
0,71948.0,3,23093.125,16.02,0.37,561.0,1,0
1,12282.0,2,1000.000,11.14,0.08,504.0,0,1
2,12438.0,0,5500.000,12.87,0.37,635.0,1,0
3,79753.0,3,23093.125,15.23,0.37,675.0,1,0
4,66135.0,3,23093.125,14.27,0.37,586.0,1,0
...,...,...,...,...,...,...,...,...
44995,47971.0,3,15000.000,15.66,0.31,645.0,1,0
44996,65800.0,3,9000.000,14.07,0.14,621.0,1,0
44997,56942.0,3,2771.000,10.02,0.05,668.0,1,0
44998,33164.0,3,12000.000,13.23,0.36,604.0,1,0


In [201]:
indep=df1.drop('loan_status', axis=1)
dep=df1['loan_status']

In [203]:
X_train, X_test, y_train, y_test = train_test_split(indep, dep, stratify=dep, test_size=0.2, random_state=42)

print("Before Resampling:", y_train.value_counts())

Before Resampling: loan_status
0    28000
1     8000
Name: count, dtype: int64


In [204]:
from imblearn.combine import SMOTEENN

smoteenn = SMOTEENN(random_state=42)
X_train_smoteenn, y_train_smoteenn = smoteenn.fit_resample(X_train, y_train)

print("After SMOTEENN:", y_train_smoteenn.value_counts())

After SMOTEENN: loan_status
1    19491
0    17909
Name: count, dtype: int64


In [205]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train_smoteenn)
X_test=scaler.transform(X_test)

In [206]:
import pickle
filename='Scaler.pkl'
pickle.dump(scaler,open(filename,'wb'))

## Logistic Regression

In [207]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['lbfgs', 'liblinear'],
             'penalty':['l2']} 

grid_lr = GridSearchCV(LogisticRegression(class_weight="balanced",max_iter=1000), param_grid, refit = True,cv=3, scoring="f1", n_jobs=-1, verbose=1)

grid_lr.fit(X_train, y_train_smoteenn)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


,estimator,LogisticRegre...max_iter=1000)
,param_grid,"{'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [238]:
y_pred=grid_lr.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.97      0.86      0.91      7000
           1       0.64      0.89      0.75      2000

    accuracy                           0.87      9000
   macro avg       0.80      0.88      0.83      9000
weighted avg       0.89      0.87      0.87      9000



In [240]:
from sklearn.metrics import f1_score
best_lr = grid_lr.best_estimator_

y_proba = best_lr.predict_proba(X_test)[:, 1]

best_t = 0.5
best_f1 = 0

for t in [0.5, 0.6, 0.7, 0.8, 0.9]:
    y_pred_t = (y_proba >= t).astype(int)
    f1 = f1_score(y_test, y_pred_t)
    
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print(f"\n✅ Best threshold = {best_t:.2f} with F1 = {best_f1:.3f}")

y_pred_best = (y_proba >= best_t).astype(int)
print("\nClassification report at best threshold:\n")
print(classification_report(y_test, y_pred_best))

cm_best = confusion_matrix(y_test, y_pred_best)


✅ Best threshold = 0.70 with F1 = 0.758

Classification report at best threshold:

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      7000
           1       0.72      0.80      0.76      2000

    accuracy                           0.89      9000
   macro avg       0.83      0.86      0.84      9000
weighted avg       0.89      0.89      0.89      9000



## Decision Tree

In [210]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': [None,'sqrt','log2'],
              'splitter':['best','random'],
              'max_depth': [5, 10, 20]} 

grid_dt =GridSearchCV(DecisionTreeClassifier(class_weight='balanced', random_state=42), param_grid, refit = True, cv=3, scoring="f1", n_jobs=-1, verbose=1) 
   
# fitting the model for grid search 
grid_dt.fit(X_train, y_train_smoteenn)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'criterion': ['gini', 'entropy'], 'max_depth': [5, 10, ...], 'max_features': [None, 'sqrt', ...], 'splitter': ['best', 'random']}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'entropy'


In [241]:
y_pred=grid_dt.predict(X_test)
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test, y_pred)
# print classification report 
from sklearn.metrics import classification_report
clf_report1 = classification_report(y_test, y_pred)
print(clf_report1)

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      7000
           1       0.71      0.82      0.76      2000

    accuracy                           0.88      9000
   macro avg       0.83      0.86      0.84      9000
weighted avg       0.89      0.88      0.89      9000



In [242]:
best_dt = grid_dt.best_estimator_
y_proba = best_dt.predict_proba(X_test)[:, 1]

best_t = 0.5
best_f1 = 0

for t in [0.3, 0.4, 0.5, 0.6, 0.7,0.8,0.9]:
    y_pred_t = (y_proba >= t).astype(int)
    f1 = f1_score(y_test, y_pred_t)
    
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print(f"\n✅ Best threshold = {best_t:.2f} with F1 = {best_f1:.3f}")
y_pred_best1 = (y_proba >= best_t).astype(int)
print("\nClassification report at best threshold:\n")
print(classification_report(y_test, y_pred_best1))

cm_best1 = confusion_matrix(y_test, y_pred_best1)


✅ Best threshold = 0.80 with F1 = 0.760

Classification report at best threshold:

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      7000
           1       0.72      0.80      0.76      2000

    accuracy                           0.89      9000
   macro avg       0.83      0.86      0.84      9000
weighted avg       0.89      0.89      0.89      9000



## Random Forest

In [214]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['sqrt','log2'],
              'n_estimators':[100,200],
              'max_depth': [10, 20]}

grid_rf = GridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42), param_grid, refit = True, cv=3, scoring="f1", n_jobs=-1, verbose=1) 
   
# fitting the model for grid search 
grid_rf.fit(X_train, y_train_smoteenn)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


,estimator,RandomForestC...ndom_state=42)
,param_grid,"{'criterion': ['gini', 'entropy'], 'max_depth': [10, 20], 'max_features': ['sqrt', 'log2'], 'n_estimators': [100, 200]}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,200


In [243]:
y_pred=grid_rf.predict(X_test)
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_test, y_pred)
# print classification report 
from sklearn.metrics import classification_report
clf_report2 = classification_report(y_test, y_pred)
print(clf_report2)

              precision    recall  f1-score   support

           0       0.96      0.92      0.93      7000
           1       0.74      0.85      0.79      2000

    accuracy                           0.90      9000
   macro avg       0.85      0.88      0.86      9000
weighted avg       0.91      0.90      0.90      9000



In [244]:
best_rf = grid_rf.best_estimator_
y_proba = best_rf.predict_proba(X_test)[:, 1]

best_t = 0.5
best_f1 = 0

for t in [0.3, 0.4, 0.5, 0.6, 0.7,0.8,0.9]:
    y_pred_t = (y_proba >= t).astype(int)
    f1 = f1_score(y_test, y_pred_t)
    
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print(f"\n✅ Best threshold = {best_t:.2f} with F1 = {best_f1:.3f}")
y_pred_best2 = (y_proba >= best_t).astype(int)
print("\nClassification report at best threshold:\n")
print(classification_report(y_test, y_pred_best2))

cm_best2 = confusion_matrix(y_test, y_pred_best2)


✅ Best threshold = 0.60 with F1 = 0.797

Classification report at best threshold:

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      7000
           1       0.79      0.80      0.80      2000

    accuracy                           0.91      9000
   macro avg       0.87      0.87      0.87      9000
weighted avg       0.91      0.91      0.91      9000



## KNN

In [217]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance']
}

grid_knn = GridSearchCV(KNeighborsClassifier(n_neighbors=5), param_grid_knn, refit = True, cv=3, scoring="f1", n_jobs=-1, verbose=1)

grid_knn.fit(X_train, y_train_smoteenn)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


,estimator,KNeighborsClassifier()
,param_grid,"{'n_neighbors': [3, 5, ...], 'weights': ['uniform', 'distance']}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_neighbors,3


In [245]:
y_pred=grid_knn.predict(X_test)
from sklearn.metrics import confusion_matrix
cm3 = confusion_matrix(y_test, y_pred)
# print classification report 
from sklearn.metrics import classification_report
clf_report3 = classification_report(y_test, y_pred)
print(clf_report3)

              precision    recall  f1-score   support

           0       0.95      0.88      0.92      7000
           1       0.67      0.85      0.75      2000

    accuracy                           0.87      9000
   macro avg       0.81      0.86      0.83      9000
weighted avg       0.89      0.87      0.88      9000



In [246]:
best_knn = grid_knn.best_estimator_
y_proba = best_knn.predict_proba(X_test)[:, 1]

best_t = 0.5
best_f1 = 0

for t in [0.3, 0.4, 0.5, 0.6, 0.7,0.8,0.9]:
    y_pred_t = (y_proba >= t).astype(int)
    f1 = f1_score(y_test, y_pred_t)
    
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print(f"\n✅ Best threshold = {best_t:.2f} with F1 = {best_f1:.3f}")
y_pred_best3 = (y_proba >= best_t).astype(int)
print("\nClassification report at best threshold:\n")
print(classification_report(y_test, y_pred_best3))

cm_best3 = confusion_matrix(y_test, y_pred_best3)


✅ Best threshold = 0.50 with F1 = 0.748

Classification report at best threshold:

              precision    recall  f1-score   support

           0       0.95      0.88      0.92      7000
           1       0.67      0.85      0.75      2000

    accuracy                           0.87      9000
   macro avg       0.81      0.86      0.83      9000
weighted avg       0.89      0.87      0.88      9000



## XGBoost

In [220]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid_xgb = {
    'n_estimators': [200, 400],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

grid_xgb = GridSearchCV(estimator=XGBClassifier(scale_pos_weight= (y_train == 0).sum() / (y_train == 1).sum(),
        eval_metric="logloss",
        use_label_encoder=False,
        random_state=42),param_grid=param_grid_xgb,refit=True,cv=3, scoring="f1", n_jobs=-1, verbose=1)

grid_xgb.fit(X_train, y_train_smoteenn)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


,estimator,"XGBClassifier...ree=None, ...)"
,param_grid,"{'learning_rate': [0.01, 0.05, ...], 'max_depth': [3, 5, ...], 'n_estimators': [200, 400], 'subsample': [0.8, 1.0]}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,objective,'binary:logistic'


In [247]:
y_pred = grid_xgb.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm4 = confusion_matrix(y_test, y_pred)
# print classification report 
from sklearn.metrics import classification_report
clf_report4 = classification_report(y_test, y_pred)
print(clf_report4)

              precision    recall  f1-score   support

           0       0.96      0.89      0.93      7000
           1       0.71      0.88      0.78      2000

    accuracy                           0.89      9000
   macro avg       0.83      0.89      0.86      9000
weighted avg       0.91      0.89      0.90      9000



In [248]:
best_xgb = grid_xgb.best_estimator_
y_proba = best_xgb.predict_proba(X_test)[:, 1]

best_t = 0.5
best_f1 = 0

for t in [0.3, 0.4, 0.5, 0.6, 0.7,0.8,0.9]:
    y_pred_t = (y_proba >= t).astype(int)
    f1 = f1_score(y_test, y_pred_t)
    
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print(f"\n✅ Best threshold = {best_t:.2f} with F1 = {best_f1:.3f}")
y_pred_best4 = (y_proba >= best_t).astype(int)
print("\nClassification report at best threshold:\n")
print(classification_report(y_test, y_pred_best4))

cm_best4 = confusion_matrix(y_test, y_pred_best4)


✅ Best threshold = 0.80 with F1 = 0.798

Classification report at best threshold:

              precision    recall  f1-score   support

           0       0.95      0.94      0.94      7000
           1       0.78      0.81      0.80      2000

    accuracy                           0.91      9000
   macro avg       0.86      0.87      0.87      9000
weighted avg       0.91      0.91      0.91      9000



In [234]:
import pickle
with open('best_xgb_model.pkl', 'wb') as f:
    pickle.dump({'model': best_xgb, 'threshold': best_t}, f)